In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!nvidia-smi

Sat Jul 31 06:18:23 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Light GBM을 colab에서 gpu로 돌리기 위한 절차

In [5]:
# After running
!git clone --recursive https://github.com/Microsoft/LightGBM
%cd /content/LightGBM
!mkdir build
!cmake -DUSE_GPU=1
!make -j$(nproc)
!sudo apt-get -y install python-pip
!sudo -H pip install setuptools pandas numpy scipy scikit-learn -U
%cd /content/LightGBM/python-package/
!sudo python setup.py install --precompile


#You can run this oneliner which will build and compile LightGBM with GPU enabled in colab:
# ! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;   

Cloning into 'LightGBM'...
remote: Enumerating objects: 22799, done.
remote: Counting objects: 100% (478/478), done.
remote: Compressing objects: 100% (273/273), done.
remote: Total 22799 (delta 256), reused 351 (delta 190), pack-reused 22321
Receiving objects: 100% (22799/22799), 17.85 MiB | 20.82 MiB/s, done.
Resolving deltas: 100% (16647/16647), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'external_libs/compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'external_libs/eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/content/LightGBM/python-package/LightGBM/external_libs/compute'...
remote: Enumerating objects: 21731, done.        
remote: Counting objects: 100% (3/3), 

## 카이 설치(설치 오류시 런타임 다시시작 해야함)

In [16]:
!git clone https://github.com/kakao/khaiii.git

fatal: destination path 'khaiii' already exists and is not an empty directory.


In [17]:
!pip install cmake

In [18]:
!mkdir build

mkdir: cannot create directory ‘build’: File exists


In [19]:
!cd build && cmake /content/khaiii

CMake Error: The source directory "/content/khaiii" does not exist.
Specify --help for usage, or press the help button on the CMake GUI.


In [20]:
!cd /content/build/ && make all

/bin/bash: line 0: cd: /content/build/: No such file or directory


In [21]:
!cd /content/build/ && make resource

/bin/bash: line 0: cd: /content/build/: No such file or directory


In [22]:
!cd /content/build && make install

/bin/bash: line 0: cd: /content/build: No such file or directory


In [23]:
!cd /content/build && make package_python

/bin/bash: line 0: cd: /content/build: No such file or directory


In [24]:
!pip install /content/build/package_python

ERROR: Invalid requirement: '/content/build/package_python'
Hint: It looks like a path. File '/content/build/package_python' does not exist.


In [15]:
from khaiii import KhaiiiApi
khai = KhaiiiApi()

wordss = khai.analyze("흔히 쓰는 넓은 의미로서의 나무는, 위로 어느 정도 이상 높이 자라며 잎이나 줄기가 달린 길다란 나무 기둥이 있는 식물을 뜻한다. ")
a = []
for word in wordss:
    for morph in word.morphs:
        print(morph.tag)
        a.append((morph.lex, morph.tag))


ImportError: ignored

## Optuna 설치

In [ ]:
pip install optuna

## Word2vec 설치

In [ ]:
pip install --upgrade gensim

In [ ]:
pip install python-Levenshtein

In [ ]:
pip install word2vec

## Bayesian-opiomizeation 설치

In [ ]:
pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp37-none-any.whl size=11686 sha256=9942fff501770857e29be931f15fb6454b156a3f22ea561e3fb6399a22387784
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization


In [ ]:
pip install wordcloud

In [ ]:
import numpy as np
import pandas as pd
import os
import re
import json
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns

from gensim.models import Word2Vec
from khaiii import KhaiiiApi


from sklearn.metrics import accuracy_score, log_loss, f1_score
from sklearn.model_selection import train_test_split, KFold

import lightgbm as lgbm
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

import optuna 
from optuna import Trial, visualization
from optuna.samplers import TPESampler


from tqdm import tqdm



# 그래프를 출력창에서 바로 볼 수 있게함
%matplotlib inline 


## 데이터 준비

In [ ]:
#################
STOPWORDSPATH ="/content/drive/MyDrive/Colab Notebooks/stopwords.txt"
#################
## Import DATA, submission file
train = pd.read_csv("/content/drive/MyDrive/DACON/topic_classification/train_data.csv", error_bad_lines=False  )
test = pd.read_csv("/content/drive/MyDrive/DACON/topic_classification/test_data.csv",error_bad_lines=False)
submission = pd.read_csv("/content/drive/MyDrive/DACON/topic_classification/sample_submission.csv",error_bad_lines=False)
topic_dict = pd.read_csv("/content/drive/MyDrive/DACON/topic_classification/topic_dict.csv",error_bad_lines=False)

## 전처리(불용어 등)

In [ ]:


## preprocessing
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

def clean_punc(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text.strip()

cleaned_train_corpus = []
cleaned_test_corpus = []

for sent in train['title']:
    cleaned_train_corpus.append(clean_punc(sent, punct, punct_mapping))
    
for sent in test['title']:
    cleaned_test_corpus.append(clean_punc(sent, punct, punct_mapping))
    
"""

1차 전처리 후 반환된 List
cleaned_train_corpus 
cleaned_test_corpus 

"""


def clean_text(texts):
    corpus = []
    for i in range(0, len(texts)):
        texts[i] = texts[i].replace("外人","외국인")
        texts[i] = texts[i].replace("日","일본")
        texts[i] = texts[i].replace("美","미국")
        texts[i] = texts[i].replace("北","북한")
        texts[i] = texts[i].replace("英","영국")
        texts[i] = texts[i].replace("中","중국")
        texts[i] = texts[i].replace("與","여당")
        texts[i] = texts[i].replace("靑","청와대")
        texts[i] = texts[i].replace("野","야당")
        texts[i] = texts[i].replace("伊","이탈리아")
        texts[i] = texts[i].replace("韓","한국")
        texts[i] = texts[i].replace("南","한국")
        texts[i] = texts[i].replace("獨","독일")
        texts[i] = texts[i].replace("佛","프랑스")
        texts[i] = texts[i].replace("檢","검찰")
        texts[i] = texts[i].replace("銀","은행")
        texts[i] = texts[i].replace("亞","아시아")
        texts[i] = texts[i].replace("人","사람")
        texts[i] = texts[i].replace("孫","손혜원")
        texts[i] = texts[i].replace("企","기업")
        texts[i] = texts[i].replace("前","이전")
        texts[i] = texts[i].replace("反","반대")
        texts[i] = texts[i].replace("安","안철수")
        texts[i] = texts[i].replace("展","전시회")
        texts[i] = texts[i].replace("故","사망")
        texts[i] = texts[i].replace("文","문재인")
        texts[i] = texts[i].replace("新","새로운")
        texts[i] = texts[i].replace("曺","조국")
        texts[i] = texts[i].replace("朴","박정치인")
        texts[i] = texts[i].replace("株","주식")
        texts[i] = texts[i].replace("男","남자")
        texts[i] = texts[i].replace("硏","연구")
        texts[i] = texts[i].replace("車","자동차")
        texts[i] = texts[i].replace("軍","군대")
        texts[i] = texts[i].replace("重","중공업")
        texts[i] = texts[i].replace("R&D","연구개발")
        texts[i] = texts[i].replace("문정부","문재인정부")
        


        
        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(texts[i])) #remove punctuation
        review = re.sub(r'\d+','', str(review))# remove number
        review = review.lower() #lower case
        review = re.sub(r'\s+', ' ', review) #remove extra space
        review = re.sub(r'<[^>]+>','',review) #remove Html tags
        review = re.sub(r'\s+', ' ', review) #remove spaces
        review = re.sub(r"^\s+", '', review) #remove space from start
        review = re.sub(r'\s+$', '', review) #remove space from the end
        review = re.sub("[一-龥]",'', review)
        corpus.append(review)
    return corpus

basic_preprocessed_train_corpus = clean_text(cleaned_train_corpus)
basic_preprocessed_test_corpus = clean_text(cleaned_test_corpus)


"""

2차 전처리 후 반환된 List
basic_preprocessed_train_corpus = clean_text(cleaned_train_corpus)
basic_preprocessed_test_corpus = clean_text(cleaned_test_corpus)


"""

'\n\n2차 전처리 후 반환된 List\nbasic_preprocessed_train_corpus = clean_text(cleaned_train_corpus)\nbasic_preprocessed_test_corpus = clean_text(cleaned_test_corpus)\n\n\n'

In [ ]:
test__ = "잡초(雜草, 영어: weed) 혹은 잡풀은 인간이 농경생활을 시작하면서 발생한 것으로 인간에 의해 재배되지 않고 저절로 나서 자라는 여러 가지 잡다한 풀로서 때와 장소에 적절하지 않은 식물이다. 대한민국 산림청 산림임업용어사전에 따르면 잡초는, '草本식물로서 苗圃 또는 林地에 發生해서 林業上 해로운 것. 나무의 경우에는 이것을 weed tree라고 한다.'라고 정의하며, 미국의 시인 랠프 월도 에머슨은 잡초는, '그 가치가 아직 발견되지 않는 식물들.' 이라고 했다. 하지만 정원사나 농부, 잡초 전문가들은 잡초를, '잡초는 농작물(crop)과 비교 했을때 그 가치가 조금 모자란 식물.' 이라고 평가한다.한마디로 잡초는 인간에 의해 구분된 식물집단이며, 과거에 잡초였다가 나중에 숨은 가치를 인정받아 농작물로 인정받기도 한다."

for y in khai.analyze(test__):
    for z in y.morphs:
        print("Token : {} -> {}".format(z.lex, z.tag))
        


Token : 잡초 -> NNG
Token : ( -> SS
Token : 雜草 -> SH
Token : , -> SP
Token : 영어 -> NNP
Token : : -> SP
Token : weed -> SL
Token : ) -> SS
Token : 혹은 -> MAG
Token : 잡풀 -> NNG
Token : 은 -> ETM
Token : 인간 -> NNG
Token : 이 -> JKS
Token : 농경 -> NNG
Token : 생활 -> NNG
Token : 을 -> JKO
Token : 시작 -> NNG
Token : 하 -> XSV
Token : 면서 -> EC
Token : 발생 -> NNG
Token : 하 -> XSV
Token : ㄴ -> ETM
Token : 것 -> NNB
Token : 으로 -> JKB
Token : 인간 -> NNG
Token : 에 -> JKB
Token : 의하 -> VV
Token : 여 -> EC
Token : 재배 -> NNG
Token : 되 -> XSV
Token : 지 -> EC
Token : 않 -> VX
Token : 고 -> EC
Token : 저절로 -> MAG
Token : 나 -> VV
Token : 아서 -> EC
Token : 자라 -> VV
Token : 는 -> ETM
Token : 여러 -> MM
Token : 가지 -> NNB
Token : 잡다 -> XR
Token : 하 -> XSA
Token : ㄴ -> ETM
Token : 풀 -> NNG
Token : 로서 -> JKB
Token : 때 -> NNG
Token : 와 -> JC
Token : 장소 -> NNG
Token : 에 -> JKB
Token : 적절 -> XR
Token : 하 -> XSA
Token : 지 -> EC
Token : 않 -> VX
Token : 은 -> ETM
Token : 식물 -> NNG
Token : 이 -> VCP
Token : 다 -> EF
Token : . -> SF
Token : 

# ! [카이 품사 종류](https://t1.daumcdn.net/thumb/R1280x0/?fname=http://t1.daumcdn.net/brunch/service/user/1oU7/image/7X4mqOJLRdhThEvJJUsAacRstXY.png)

In [ ]:
"""

2차 전처리 후 반환된 List
basic_preprocessed_train_corpus = clean_text(cleaned_train_corpus)
basic_preprocessed_test_corpus = clean_text(cleaned_test_corpus)


"""

stopwords = []
with open(STOPWORDSPATH) as f:
    for line in f:
        stopwords.append(line.strip())

removed_stopword_train_corpus = []
removed_stopword_test_corpus = []

for tagged in tqdm(basic_preprocessed_train_corpus):
    temp = []
    for tagg in khai.analyze(tagged):   
        for tag in tagg.morphs:
            
            #일반명사, 고유명사, 동사, 형용사, 긍정지정사, 부정 지정사, 관형사, 일반부사, 체언접두사, 외국어, 한자
            if tag.lex in stopwords or tag.tag not in ["NNG", "NNP", "VV", "VA", "VCP", "VCN", "MM", "MAG", "XPN", "SL", "SH"]:
                continue
            temp.append(tag.lex)

    removed_stopword_train_corpus.append(' '.join(temp))
    
for tagged in tqdm( basic_preprocessed_test_corpus):
    temp = []
    for tagg in khai.analyze(tagged):   
        for tag in tagg.morphs:
            
            #일반명사, 고유명사, 동사, 형용사, 긍정지정사, 부정 지정사, 관형사, 일반부사, 체언접두사, 외국어, 한자
            if tag.lex in stopwords or tag.tag not in ["NNG", "NNP", "VV", "VA", "VCP", "VCN", "MM", "MAG", "XPN", "SL", "SH"]:
                continue
            temp.append(tag.lex)


    removed_stopword_test_corpus.append(' '.join(temp))



"""

3차 전처리후 반환된 List
removed_stopword_train_corpus = []
removed_stopword_test_corpus = []

"""

"""   
##########################################
###############최종 전처리###################
##########################################
"""
train_text = removed_stopword_train_corpus
test_text = removed_stopword_test_corpus
train_label = np.asarray(train.topic_idx)
"""
##########################################
##########################################
"""

100%|██████████| 9131/9131 [00:08<00:00, 1034.98it/s]


'\n##########################################\n##########################################\n'

In [ ]:
train['clear_title'] = train_text
test['clear_title'] = test_text

In [ ]:
train

,index,title,topic_idx,clear_title
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4,인천 핀란드 항공기 결항 휴가철 여행객 분통
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4,실리콘밸리 넘어서 구글 조원 들이 미국 전역 거점
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4,이란 외무 긴장 완화 해결책 미국 경제 전쟁 멈추
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4,nyt 클린턴 측근한국 기업 특수 관계 조명 공 사 맞물리 종합
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4,시진핑 트럼 프 중미 무역 협상 조속 타결 희망
...,...,...,...,...
45649,45649,KB금융 미국 IB 스티펠과 제휴…선진국 시장 공략,1,kb 금융 미국 ib 스티펠 제휴 선진국 시장 공략
45650,45650,1보 서울시교육청 신종코로나 확산에 개학 연기·휴업 검토,2,보 서울시교육청 신종코 로 확산 개학 연기 휴업 검토
45651,45651,게시판 키움증권 2020 키움 영웅전 실전투자대회,1,게시판 키움 증권 키움 영웅전 실전 투자 대회
45652,45652,답변하는 배기동 국립중앙박물관장,2,답변 배기동 국립중앙박물관 장


In [ ]:
def split(text):
    tokens_ko = text.split()
    return tokens_ko

tfidf_vect = TfidfVectorizer(tokenizer=split)
tfidf_vect.fit(train['clear_title'])
tfidf_matrix_train = tfidf_vect.transform(train['clear_title'])
tfidf_matrix_test = tfidf_vect.transform(test['clear_title'])

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:489: UserWarning:

The parameter 'token_pattern' will not be used since 'tokenizer' is not None'



In [ ]:
 !nvidia-smi 

Thu Jul 15 15:21:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
rand_num = 977

def objective(trial: Trial) -> float:
    params_lgb = {
        "random_state": rand_num,
        "verbosity": -1,
        "learning_rate": trial.suggest_float("learning_rate", 0.05, 0.15),
        "n_estimators": trial.suggest_int('n_estimators', 700, 3000), # 반복 수행하는 트리의 개수 너무 크면 overfitting 발생
        "objective": "multiclass",
        "boosting_type" : trial.suggest_categorical("boosting_type", ["gbdt", "dart","rf"]),
        "metric": "multi_error",
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 3e-5), # L1 규제(과적합 제어)
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 9e-2), # L2 규제(과적합 제어)
        "max_depth": trial.suggest_int("max_depth", 1, 10), # 트리의 최대 깊이 ★★★★★ default는 깊이제한 없음
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 5),
        "num_leaves": trial.suggest_int("num_leaves", 2, 128), # 하나의 트리가 가질 수 있는 최대 leaf 개수 ★★★★★ num_leaves는 2^(max_depth)보다 작아야 함
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0), # 개별 트리를 학습할 때마다 무작위로 선택하는 feature의 비율을 제어
        "subsample": trial.suggest_float("subsample", 0.5, 1.0), # 과적합을 제어하기 위해 데이터를 샘플링 하는 비율 ★★★★★ ; 0.5면 전체 data에 절반을 tree 생성에 사용
        "subsample_freq": trial.suggest_int("subsample_freq", 1, 10),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 2000), # 최종 결정 클래스인 leaf node가 되기 위한 최소한의 데이터 개체 수(과적합 제어) ★★★★★
        "max_bin": trial.suggest_int("max_bin", 200, 500),
        'device' : 'gpu',
    }
    X_train, X_valid, y_train, y_valid = train_test_split(tfidf_matrix_train, train['topic_idx'], random_state=rand_num, test_size=0.2)

    model = LGBMClassifier(**params_lgb)
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        early_stopping_rounds=100,
        verbose=False,
    )

    lgb_pred = model.predict_proba(X_valid)
    log_score = log_loss(y_valid, lgb_pred)
    
    return log_score

In [ ]:
sampler = TPESampler(seed=rand_num)
study = optuna.create_study(
    study_name="lgbm_parameter_opt",
    direction="minimize",
    sampler=sampler,
)
study.optimize(objective, n_trials=30)
print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

[I 2021-07-15 15:26:36,315] A new study created in memory with name: lgbm_parameter_opt
/usr/local/lib/python3.7/dist-packages/lightgbm/callback.py:182: UserWarning:

Early stopping is not available in dart mode

[I 2021-07-15 15:29:42,898] Trial 0 finished with value: 1.1996834197985433 and parameters: {'learning_rate': 0.05584820734678016, 'n_estimators': 1601, 'boosting_type': 'dart', 'reg_alpha': 5.916112781249445e-06, 'reg_lambda': 0.03349975665098382, 'max_depth': 1, 'min_child_weight': 1, 'num_leaves': 123, 'colsample_bytree': 0.9964087446479873, 'subsample': 0.6923698093621811, 'subsample_freq': 4, 'min_child_samples': 145, 'max_bin': 297}. Best is trial 0 with value: 1.1996834197985433.
[I 2021-07-15 15:29:44,952] Trial 1 finished with value: 1.93578198812334 and parameters: {'learning_rate': 0.05718060639600664, 'n_estimators': 1596, 'boosting_type': 'gbdt', 'reg_alpha': 2.018843144208835e-05, 'reg_lambda': 0.0005296343555110703, 'max_depth': 6, 'min_child_weight': 3, 'num_le

Best Score: 0.6557186233315969
Best trial: {'learning_rate': 0.07746386542679269, 'n_estimators': 2421, 'boosting_type': 'gbdt', 'reg_alpha': 1.6927348901059054e-05, 'reg_lambda': 0.07490075798579968, 'max_depth': 4, 'min_child_weight': 4, 'num_leaves': 22, 'colsample_bytree': 0.6329624658030231, 'subsample': 0.9975382210447198, 'subsample_freq': 8, 'min_child_samples': 18, 'max_bin': 350}


In [ ]:
optuna.visualization.plot_optimization_history(study).show()
#plot_parallel_coordinate: interactively visualizes the hyperparameters and scores
optuna.visualization.plot_parallel_coordinate(study).show()
'''plot_slice: shows the evolution of the search. You can see where in the hyperparameter space your search
went and which parts of the space were explored more.'''
optuna.visualization.plot_slice(study).show()

#Visualize parameter importances.
optuna.visualization.plot_param_importances(study).show()
#Visualize empirical distribution function
optuna.visualization.plot_edf(study).show()

In [ ]:
best_params = {}
best_params.update(study.best_params)
best_params

{'boosting_type': 'gbdt',
 'colsample_bytree': 0.6329624658030231,
 'learning_rate': 0.07746386542679269,
 'max_bin': 350,
 'max_depth': 4,
 'min_child_samples': 18,
 'min_child_weight': 4,
 'n_estimators': 2421,
 'num_leaves': 22,
 'reg_alpha': 1.6927348901059054e-05,
 'reg_lambda': 0.07490075798579968,
 'subsample': 0.9975382210447198,
 'subsample_freq': 8}

In [ ]:
# tfidf_matrix_train = tfidf_vect.fit_transform(t)
# tfidf_matrix_test = tfidf_vect.transform(test['clear_title'])

mean_submission = pd.read_csv("/content/drive/MyDrive/DACON/topic_classification/sample_submission.csv",error_bad_lines=False)

cls_model = LGBMClassifier(**best_params)

folds = KFold(n_splits = 5, shuffle = True, random_state = rand_num)
best_result_list=[]
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train)) :
    print(n_fold)

    X_train, y_train = tfidf_matrix_train[trn_idx], train.topic_idx[trn_idx] 
    X_valid, y_valid = tfidf_matrix_train[val_idx], train.topic_idx[val_idx]
    
    cls_model.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
         early_stopping_rounds=100,
        verbose=100
    )
    lgbm_pred = cls_model.predict_proba(tfidf_matrix_test)
    # mean_submission = mean_submission.append({f'{n_fold}':lgbm_pred }, ignore_index=True)
    best_result_list.append(lgbm_pred)
    # mean_submission = mean_submission.append(lgbm_pred)
    # mean_submission['topic_idx'] = lgbm_pred
# #     sub_preds += model.predict(x_test) / folds.n_splits
#     oof_preds[val_idx] = model.predict_prova(val_x)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning:

The error_bad_lines argument has been deprecated and will be removed in a future version.





0
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.999646	valid_1's multi_logloss: 1.02278
[200]	training's multi_logloss: 0.828497	valid_1's multi_logloss: 0.870025
[300]	training's multi_logloss: 0.736075	valid_1's multi_logloss: 0.793745
[400]	training's multi_logloss: 0.6753	valid_1's multi_logloss: 0.747108
[500]	training's multi_logloss: 0.631635	valid_1's multi_logloss: 0.717109
[600]	training's multi_logloss: 0.598101	valid_1's multi_logloss: 0.695762
[700]	training's multi_logloss: 0.571574	valid_1's multi_logloss: 0.680285
[800]	training's multi_logloss: 0.549475	valid_1's multi_logloss: 0.66906
[900]	training's multi_logloss: 0.530891	valid_1's multi_logloss: 0.660329
[1000]	training's multi_logloss: 0.514712	valid_1's multi_logloss: 0.65453
[1100]	training's multi_logloss: 0.5006	valid_1's multi_logloss: 0.649491
[1200]	training's multi_logloss: 0.488353	valid_1's multi_logloss: 0.645792
[1300]	training's multi_logloss: 0.47712

In [ ]:
a = best_result_list[0] + best_result_list[1] + best_result_list[2] + best_result_list[3] + best_result_list[4]
a = a/5
a

array([[0.08994419, 0.07487795, 0.60941842, ..., 0.05463454, 0.01559412,
        0.03249672],
       [0.00647496, 0.01131499, 0.02668293, ..., 0.00866866, 0.00668844,
        0.01246719],
       [0.22330218, 0.16143568, 0.24170509, ..., 0.09249782, 0.09892795,
        0.115186  ],
       ...,
       [0.05707994, 0.01464248, 0.10963638, ..., 0.12745003, 0.12045781,
        0.01716644],
       [0.48076864, 0.05012882, 0.29312425, ..., 0.00575763, 0.00242909,
        0.012141  ],
       [0.06622626, 0.05541676, 0.54586048, ..., 0.06481221, 0.04586674,
        0.12307441]])

In [ ]:
pred = np.argmax(a, axis = 1)
submission.topic_idx = pred
submission

,index,topic_idx
0,45654,2
1,45655,3
2,45656,2
3,45657,2
4,45658,3
...,...,...
9126,54780,3
9127,54781,2
9128,54782,3
9129,54783,0


In [ ]:
submission.to_csv('/content/LightGBM_baseline.csv',index = False)
# submission.to_csv('bert_baseline_1.csv',index = True)

In [ ]:
!pip install /content/dacon_submit_api-0.0.4-py3-none-any.whl

Processing ./dacon_submit_api-0.0.4-py3-none-any.whl


In [ ]:
from dacon_submit_api import dacon_submit_api 

result = dacon_submit_api.post_submission_file(
'/content/LightGBM_baseline.csv', # 파일경로
'13d2c7d751f562a69b53e42c532395982f1a2e09eab2181a7c788317e3d8a792',  # 개인토큰
'235747', # 대회 id
'Healthy Guys',  # 팀이름
'lgbm') # 노트

{'isSubmitted': True, 'detail': 'Success'}
